<a href="https://colab.research.google.com/github/AmmarNassan/Data-Science-Projects/blob/main/6_train_roberta_from_scratch_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh |  bash
!apt-get install -y --allow-unauthenticated git-lfs

In [ ]:
%%capture 
! pip install transformers
! pip install datasets
! pip install huggingface_hub

In [ ]:
import transformers
print(transformers.__version__)

4.20.1


In [ ]:
import numpy as np 
import pandas as pd 
import os 
from pathlib import Path
from scipy.stats import pearsonr
from datasets import load_dataset
from transformers import TFAutoModel, AutoTokenizer

import tensorflow as tf
from transformers import AdamWeightDecay

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [ ]:
paths = [str(x) for x in Path('/content/drive/MyDrive/Roberta-Data-Tokenizer').glob('**/*.txt')]
len(paths)

2

In [ ]:
paths=paths[0:1]
len(paths)

1

In [ ]:
with open(paths[0], 'r', encoding='utf-8') as f:
    lines = f.read()
    
lines[0:500]

'أهرب من ذاكرتي باحثاً عن صورة ضاحكة متفائلة استقبل بها العام الجديد، فتحاصرني دموع الأطفال المفزوعة، وقد اختزنت الذاكرة ألواناً منها بالغة القسوة والايذاء، في جنين وبيت لحم ونابلس ورفح وقلقيلية وكل الأرض الفلسطينية النازفة المفجوعة المطاردة بصواريخ وأحقاد واطماع شارون وحلفائه من أعداء العدل والشرعية والقانون وحقوق الانسان، وتحاصرني الوجوه المكسورة، والاستغاثات الخرساء، والرؤوس المغطاة بالأكياس، في يوميات العراق المثقلة بالجوع والقهر والدم والنار.                                                  '

In [ ]:
#datasets = load_dataset("text", data_files={"train": paths, "validation":  ['/content/drive/MyDrive/Roberta-Data-Tokenizer/text_1.txt']})

In [ ]:
from datasets import load_dataset

datasets = load_dataset("wikitext", "wikitext-2-raw-v1")

Reusing dataset wikitext (/root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from tokenizers import Tokenizer,pre_tokenizers
from transformers import PreTrainedTokenizerFast 

In [ ]:
#os.environ["TOKENIZERS_PARALLELISM"] = "false"
#tokenizer=PreTrainedTokenizerFast.from_pretrained('/content/drive/MyDrive/Roberta-Data-Tokenizer/mytokenizer', TOKENIZERS_PARALLELISM=False)
#tokenizer

In [ ]:
#if not os.path.exists('./Araroberta-retrained'):
    #os.mkdir('./Araroberta-retrained')

#tokenizer.save_pretrained('Araroberta-retrained')

In [ ]:
model_checkpoint = "microsoft/deberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=["text"]
)

Parameter 'function'=<function tokenize_function at 0x7f13f4eb6200> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

...

Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (638 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 64

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/10 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/10 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' Chronicles III outside Japan, is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable. Released in January 2011 in Japan, it is the third game in the Valkyria series. Employing the same fusion of tactical and real @-@ time gameplay as its predecessors,'

In [ ]:
from transformers import RobertaConfig, RobertaModel
from transformers import AutoConfig, TFAutoModelForMaskedLM

Config = RobertaConfig()
#model = TFAutoModelForMaskedLM.from_config(Config)

In [ ]:
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/529M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDebertaForMaskedLM.

Some layers of TFDebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
learning_rate = 2e-5
weight_decay = 0.01
push_to_hub_model_id = "Araroberta-retrained-id"

In [ ]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf"
)

In [ ]:
train_set = lm_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=64,
    collate_fn=data_collator,
)
validation_set = lm_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=64,
    collate_fn=data_collator,
)

In [ ]:
model.fit(train_set, validation_data=validation_set, epochs=2)

Epoch 1/2
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [ ]:
model.save_pretrained('./Araroberta-retrained')  # and don't forget to save model!

In [ ]:
from transformers import pipeline

fill = pipeline('fill-mask', model='Araroberta-retrained', tokenizer='Araroberta-retrained')

In [ ]:
fill(f'الذي اجراه {fill.tokenizer.mask_token} الأوروبي') # 

https://github.com/huggingface/notebooks/blob/main/examples/language_modeling-tf.ipynb